<a href="https://colab.research.google.com/github/anshulsolanki/stock_analysis/blob/main/Nifty_Monthly_SIP_with_sell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The RSI based strategy of only buying will never beat SIP on nifty50 over a long period. The primary reason for this is that RSI remains between 0-1 , while the price can keep rising , thus an RSI of 30 at price of 4000 is good in 2011 , but the same RSI 30 in 2015 may be at price 7000 , just becuase the market has run up and you miss to invest in between.

SIP on the other hand benefits from averaging during the whole period.

In order to beat SIP we will have to include the sell in strategy

====
Results - included the sell strategy , it is giving even bad results ....the reason is although each trade was profitable , but you were buying high and selling higher....
whereas in SIP you bought at lowest becuase even if the RSI touched 10 , the price did not touch low of SIP ....

Learning is that for something which is in continous uptrend , trading will not give better profit than SIP

In [ ]:
 #Install the ta-lib -> this will be used for doign the technical analysis
def install_talib():
  url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
  ext = '0.4.0-oneiric1_amd64.deb -qO'
  !wget $url/libta-lib0_$ext libta.deb
  !wget $url/ta-lib0-dev_$ext ta.deb
  !dpkg -i libta.deb ta.deb
  !pip install ta-lib

  #addtional install
  !pip install --upgrade pandas-datareader

def install_yahoo_finance():
  # Install yfinance package.
  !pip install yfinance

install_talib()
install_yahoo_finance()

Selecting previously unselected package libta-lib0.
(Reading database ... 128126 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.9) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.0/272.0 KB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ta-lib: filename=TA_Lib-0.4.25-cp38-cp38-linux_x86_64.whl size=2510152 sha256=a41c3d78ee1972f6a02e3e29e5c33

In [ ]:
!pip install backtesting
from backtesting import Backtest , Strategy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173819 sha256=ac12c2217d772e6dedac88f86d5d6a9770aca727114b09f8d6046734cb036031
  Stored in directory: /root/.cache/pip/wheels/cd/5c/f7/aafe95c37f8b07f838fb0a8cb3177de23a38c09cbd10b447b8
Successfully built backtesting


/usr/local/lib/python3.8/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
import talib as ta
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
#lets test the weekly strategy
#get monthly data for Nifty 50
stock_name = '^NSEI'
data = yf.download(stock_name,'2010-01-01','2023-01-01',interval='1wk')
data['RSI'] = ta.RSI(data['Close'],8)

[*********************100%***********************]  1 of 1 completed


In [ ]:
SIP_installments = 10000

def capital_to_deploy( corpus, RSI ):
  if(RSI > 40):
    return 0
  if (RSI <= 40 and RSI > 30 ):
    return corpus*.1
  if (RSI <= 30 and RSI > 20 ):
    #print ( 'increase - 2')
    return corpus*.4 #+ SIP_installments*2
  if (RSI <= 20 ):
    #print ( 'increase - 4')
    return corpus*.7 #+ SIP_installments*4

def capital_to_takeout( corpus, RSI ):
  if(RSI < 70):
    return 0
  # if (RSI >= 60 and RSI < 70 ):
  #   return corpus*.2
  if (RSI >= 70 and RSI < 80 ):
    return corpus*.3
  if (RSI >= 80 ):
    return corpus*.5

In [ ]:
#RSI on weekly basis for 8 week period less than 40 - buy

Captial = 0
num_shares = 0
profit_taken = 0
strategy_trades = 0

normal_sip_share = 0
sip_trades = 0

profit_booked = 0

for index, row in data.iterrows():
  Captial += SIP_installments

  #check to buy
  if (row['RSI'] < 40 ):
    to_deploy = capital_to_deploy(Captial, round(row['RSI']))

    num_shares += round(to_deploy/row['Close'])
    Captial -= to_deploy
    strategy_trades +=1
    print ('Buy->', index , round(row['Close']) , round(to_deploy) , round(row['RSI']), 'shares=',num_shares , 'corpus=',Captial )

  #check to sell
  if (row['RSI'] > 70  and num_shares > 2 ):
    #print ( 'bb', num_shares)
    share_value = row['Close'] * num_shares
    to_take = capital_to_takeout(share_value, round(row['RSI']))

    num_shares -= round(to_take/row['Close'])
    #print ( 'cc', num_shares)

    Captial += to_take
    strategy_trades +=1
    print ('Sell->',index , round(row['Close']) , round(to_deploy) , round(row['RSI']), 'shares=',num_shares,'corpus=',Captial )

  normal_sip_share += round(SIP_installments/row['Close'])
  sip_trades +=1


share_value = data.iloc[-1]['Close'] * num_shares/100000
sip_share_value = data.iloc[-1]['Close'] * normal_sip_share/100000

print ('*****************')
print('strategy numbers')
print('no. of shares', num_shares)
print ('no of trades' , strategy_trades)
print('value of shares ' , round(share_value))
print('extra cash', round(Captial/100000))
print ('*****************')
print ('SIP numbers')
print('no. of shares', normal_sip_share)
print ('no of trades', sip_trades)
print('value of shares ' , round(sip_share_value))
print ('*****************')
print ('difference' , round(share_value) - round(sip_share_value) + round(Captial/100000) )

Buy-> 2010-05-03 00:00:00 5018 18000 34 shares= 4 corpus= 162000.0
Buy-> 2010-05-17 00:00:00 4931 18200 33 shares= 8 corpus= 163800.0
Sell-> 2010-09-13 00:00:00 5885 18200 79 shares= 6 corpus= 347923.88046875
Sell-> 2010-09-20 00:00:00 6018 18200 82 shares= 3 corpus= 375978.7798828125
Sell-> 2010-09-27 00:00:00 6143 18200 84 shares= 1 corpus= 395193.87973632815
Buy-> 2011-01-10 00:00:00 5655 54519 39 shares= 11 corpus= 490674.4917626953
Buy-> 2011-01-24 00:00:00 5512 51067 35 shares= 20 corpus= 459607.0425864258
Buy-> 2011-01-31 00:00:00 5396 46961 31 shares= 29 corpus= 422646.3383277832
Buy-> 2011-02-07 00:00:00 5310 173059 29 shares= 62 corpus= 259587.80299666993
Buy-> 2011-02-14 00:00:00 5459 26959 38 shares= 67 corpus= 242629.02269700292
Buy-> 2011-02-21 00:00:00 5304 25263 33 shares= 72 corpus= 227366.1204273026
Buy-> 2011-03-14 00:00:00 5374 25737 39 shares= 77 corpus= 231629.50838457234
Buy-> 2011-05-16 00:00:00 5486 32163 40 shares= 83 corpus= 289466.55754611513
Buy-> 2011-05-2

In [ ]:
from backtesting.lib import crossover

class SIPstrategy(Strategy):
  def init(self):
    price = self.data.Close

  def next(self):
    price = self.data.Close[-1]
    position_size = round(self.equity/price)
    if (self.data.RSI < 40):
      self.position.close(portion = .0001)
      self.buy(size=.999)
      #self.cash -=10_000
      print(position_size , self.equity)


In [ ]:
#cannot do SIP mode backtesting

backtest = Backtest(data,SIPstrategy, cash= 10000, commission= .002, exclusive_orders=True )
stats = backtest.run()
print(stats)
print(stats['_trades'].to_string())
#backtest.plot()

2 10000
2 9608.236
2 9540.266400390625
2 9419.317405273438
2 9201.625705078124
2 9492.22430078125
2 10029.952405273438
2 12518.491905273437
2 12480.276102539061
2 12450.642100585936
2 12445.825898437499
2 12047.176505859374
2 11455.497505859374
1 14840.970810546874
1 15456.73780078125
1 15613.315900390624
2 12986.541201171874
1 14231.523392578125
1 13932.855806640624
1 20113.352302734376
Start                     2010-01-01 00:00:00
End                       2022-12-01 00:00:00
Duration                   4717 days 00:00:00
Exposure Time [%]                   91.025641
Equity Final [$]                 23173.644121
Equity Peak [$]                  23173.644121
Return [%]                         131.736441
Buy & Hold Return [%]              270.854487
Return (Ann.) [%]                  614.485059
Volatility (Ann.) [%]              465.797921
Sharpe Ratio                          1.31921
Sortino Ratio                       16.036593
Calmar Ratio                        28.351867
Max. Drawdo

<ipython-input-90-f731ba6ad9c6>:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  backtest = Backtest(data,SIPstrategy, cash= 10000, commission= .002, exclusive_orders=True )
